In [1]:
import numpy as np
import pandas as pd
import warnings

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as patches

warnings.filterwarnings('ignore')

In [2]:
#mapping = pd.read_csv("Mapping_csv.csv")
mapping = pd.read_csv("../../../data/essentials/md_paths_v3.csv", index_col=[0])

#reports = pd.read_csv("AnomalyReportData.csv")
reports = pd.read_excel('../../../data/essentials/Report By Category.xls', index_col=[0])

In [3]:
# Only Select Milk Chilling Units
reports = reports[(reports["Equipment Type"] == "MILK CHIILLING UNIT") | \
                  (reports["Equipment Type"] == "MILK CHILLING UNIT")]

reports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4917 entries, 287523 to 417820
Data columns (total 82 columns):
Request Status                               4917 non-null object
Created Date                                 4917 non-null datetime64[ns]
SLA Crossed                                  4917 non-null object
Priority                                     4917 non-null object
Request Type                                 4917 non-null object
Equipment Type                               4917 non-null object
Equipment Sap Code                           4917 non-null object
Equipment   Serial No                        4917 non-null object
Equipment Description                        4917 non-null object
Category                                     4917 non-null object
SubCategory                                  4917 non-null object
Major Issue                                  4917 non-null object
Description                                  4888 non-null object
SKU                 

In [4]:
# Fetch Booth Number and Client Display Number
mapping = mapping[~mapping["client_display_name"].isnull()]

mapping["booth_id"] = list(mapping['client_display_name'].apply(lambda x: x.split("|")[0].split(".")[-1]).astype(int))
mapping["num"] = list(mapping['MD_id'].apply(lambda x: abs(int(x.split("MotherDairy")[1]))))

mapping.head()

,MD_id,client_display_name,path_mcu,path_temp,path_relay,path_door,path_amb,booth_id,num
0,MotherDairy-89,Booth No. 601 | Rajouri Garden,/MotherDairy-89/Meter2/PowerBPhase,/MotherDairy-89/TemperatureSensor4/Temperature,/MotherDairy-89/Relay11/RemoteControl,/MotherDairy-89/ReedSensor15/Status,/MotherDairy-89/TemperatureSensor6/Temperature,601,89
1,MotherDairy-59,Booth No. 3 | Kailash Colony,/MotherDairy-59/Meter2/PowerBPhase,/MotherDairy-59/TemperatureSensor4/Temperature,/MotherDairy-59/Relay10/RemoteControl,/MotherDairy-59/ReedSensor15/Status,/MotherDairy-59/TemperatureSensor6/Temperature,3,59
2,MotherDairy-93,Booth No. 538 | Rohini,/MotherDairy-93/Meter2/PowerBPhase,/MotherDairy-93/TemperatureSensor4/Temperature,/MotherDairy-93/Relay10/RemoteControl,/MotherDairy-93/ReedSensor15/Status,/MotherDairy-93/TemperatureSensor6/Temperature,538,93
3,MotherDairy-119,Booth No. 656 | Pitampura,/MotherDairy-119/Meter2/PowerBPhase,/MotherDairy-119/TemperatureSensor100/Temperature,/MotherDairy-119/Relay11/RemoteControl,/MotherDairy-119/ReedSensor50/Status,NaN,656,119
4,MotherDairy-33,Booth No. 922 | Gurgaon,/MotherDairy-33/Meter2/PowerBPhase,/MotherDairy-33/TemperatureSensor4/Temperature,/MotherDairy-33/Relay10/RemoteControl,/MotherDairy-33/ReedSensor15/Status,/MotherDairy-33/TemperatureSensor5/Temperature,922,33


In [5]:
# Read Power and Temperature Data
st_streams = pd.HDFStore('../../../data/raw/streams.h5')

power = st_streams['/streams/mcu']
temp = st_streams['/streams/temp']

st_streams.close()

In [6]:
# Booths for which we have data
mds = [int(k.split('mdb')[1]) for k in power.columns.values]
booths = list(mapping[mapping["num"].isin(mds)]["booth_id"])

In [7]:
# Only select booths for which we have data
reports = reports[(reports[u'Service Location'].isin(booths)) & \
                  (reports[u'Request Status'] == 'Closed')]

reports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 723 entries, 287537 to 415951
Data columns (total 82 columns):
Request Status                               723 non-null object
Created Date                                 723 non-null datetime64[ns]
SLA Crossed                                  723 non-null object
Priority                                     723 non-null object
Request Type                                 723 non-null object
Equipment Type                               723 non-null object
Equipment Sap Code                           723 non-null object
Equipment   Serial No                        723 non-null object
Equipment Description                        723 non-null object
Category                                     723 non-null object
SubCategory                                  723 non-null object
Major Issue                                  723 non-null object
Description                                  716 non-null object
SKU                               

In [8]:
# Select only required columns
reports_sub = reports[["Service Location" , "Created Date", "Close Date", \
                       "Major Issue", "Description", "Resolution Description"]].reset_index()
reports_sub.head()

,Request No,Service Location,Created Date,Close Date,Major Issue,Description,Resolution Description
0,287537,744,2016-04-01 08:09:14.463,2016-04-02 14:42:13.420,MCU temp high,Shopkeeper complaint that his MCU is not cooli...,Ice malte by technician MCU working good
1,287912,99,2016-04-01 17:15:29.810,2016-04-03 06:49:01.713,MCU temp high,Shopkeeper complaint that his MCU is not cooling.,Ice mallt by technician MCU working good
2,288009,956,2016-04-02 08:03:46.887,2016-04-03 06:48:51.507,MCU temp high,Shopkeeper complaint that his MCU is not cooli...,Ice malte by technician MCU working good
3,288133,853,2016-04-02 10:49:50.677,2016-04-03 15:25:20.007,MCU temp high,Shopkeeper complaint that his MCU is not cooli...,Starting capictor change MCU working good
4,288445,423,2016-04-02 18:01:53.000,2016-04-03 06:47:20.933,MCU temp high,\nShopkeeper complaint that his MCU is not co...,Contecat by harmeet


In [9]:
# Output Frame
df_man = reports_sub.copy()
df_man.head()

,Request No,Service Location,Created Date,Close Date,Major Issue,Description,Resolution Description
0,287537,744,2016-04-01 08:09:14.463,2016-04-02 14:42:13.420,MCU temp high,Shopkeeper complaint that his MCU is not cooli...,Ice malte by technician MCU working good
1,287912,99,2016-04-01 17:15:29.810,2016-04-03 06:49:01.713,MCU temp high,Shopkeeper complaint that his MCU is not cooling.,Ice mallt by technician MCU working good
2,288009,956,2016-04-02 08:03:46.887,2016-04-03 06:48:51.507,MCU temp high,Shopkeeper complaint that his MCU is not cooli...,Ice malte by technician MCU working good
3,288133,853,2016-04-02 10:49:50.677,2016-04-03 15:25:20.007,MCU temp high,Shopkeeper complaint that his MCU is not cooli...,Starting capictor change MCU working good
4,288445,423,2016-04-02 18:01:53.000,2016-04-03 06:47:20.933,MCU temp high,\nShopkeeper complaint that his MCU is not co...,Contecat by harmeet


In [10]:
# Get MD_ID from Booth_ID
def get_num(booth_id):
    return int(mapping.ix[mapping["booth_id"] == booth_id, "num"].values[0])

In [11]:
plt_data = False

# Create CSV and Generate Plots
for i, r in df_man.iterrows():
    
    cr_adt = df_man.ix[i, "Created Date"]
    cl_adt = df_man.ix[i, "Close Date"]
    
    cr_dt = cr_adt - pd.to_timedelta(20, unit='d')
    cl_dt = cl_adt + pd.to_timedelta(5, unit='d')

    sindx = str(cr_dt.date())
    eindx = str(cl_dt.date())

    elapsed_time = (cl_dt.date() - cr_dt.date()).total_seconds()
    n_instances = elapsed_time / 30.0

    service_location = df_man.ix[i, "Service Location"]
    bth_id = get_num(service_location)
    bth_col = "mdb" + str(bth_id)

    # print sindx, eindx, bth_id, bth_col, elapsed_time, n_instances

    pow_sub = power.ix[sindx:eindx, bth_col]
    temp_sub = temp.ix[sindx:eindx, bth_col]

    # print pow_sub.shape

    df_man.ix[i, "start"] = cr_dt.date()
    df_man.ix[i, "stop"] = cl_dt.date()
    df_man.ix[i, "md_id"] = bth_id
    df_man.ix[i, "pow_data"] = (pow_sub.count()) / n_instances
    df_man.ix[i, "temp_data"] = (temp_sub.count()) / n_instances
    
    if (pow_sub.count() > 0) & (temp_sub.count() > 0) & (plt_data == True):
        fig, ax = plt.subplots(figsize=(16, 9))

        pow_sub.dropna().plot(ax=ax, color='blue', fontsize=20)
        ax.set_ylabel('Power (kWh)', fontsize=20, color='blue')

        ax2 = ax.twinx()
        temp_sub.dropna().plot(ax=ax2, color='red', fontsize=20)
        ax2.set_ylabel('Temperature', fontsize=20, color='red')

        ax.add_patch(
            patches.Rectangle(
                (mdates.date2num(cr_adt), 0.1),   # (x,y)
                mdates.date2num(cl_dt) - mdates.date2num(cr_adt),          # width
                3000,          # height
                facecolor="black", 
                alpha = 0.6
            )
        )

        plt.tight_layout()
        plt.savefig('../../../figures/raw/anomalies/anom' + str(i) + '_' + str(df_man.ix[i, "Request No"]) + '.pdf')

    plt.close()

df_man.head()

,Request No,Service Location,Created Date,Close Date,Major Issue,Description,Resolution Description,start,stop,md_id,pow_data,temp_data
0,287537,744,2016-04-01 08:09:14.463,2016-04-02 14:42:13.420,MCU temp high,Shopkeeper complaint that his MCU is not cooli...,Ice malte by technician MCU working good,2016-03-12,2016-04-07,66.0,0.0,0.0
1,287912,99,2016-04-01 17:15:29.810,2016-04-03 06:49:01.713,MCU temp high,Shopkeeper complaint that his MCU is not cooling.,Ice mallt by technician MCU working good,2016-03-12,2016-04-08,36.0,0.0,0.0
2,288009,956,2016-04-02 08:03:46.887,2016-04-03 06:48:51.507,MCU temp high,Shopkeeper complaint that his MCU is not cooli...,Ice malte by technician MCU working good,2016-03-13,2016-04-08,45.0,0.0,0.0
3,288133,853,2016-04-02 10:49:50.677,2016-04-03 15:25:20.007,MCU temp high,Shopkeeper complaint that his MCU is not cooli...,Starting capictor change MCU working good,2016-03-13,2016-04-08,41.0,0.0,0.0
4,288445,423,2016-04-02 18:01:53.000,2016-04-03 06:47:20.933,MCU temp high,\nShopkeeper complaint that his MCU is not co...,Contecat by harmeet,2016-03-13,2016-04-08,94.0,0.0,0.0


In [13]:
df_man.to_csv('../../../data/essentials/anomalies.csv', index=False, encoding='utf-8')